In [1]:
import torch
import torch.nn as nn
from tqdm import tqdm
import re

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long).to(device)

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMDEDDING_DIM = 10

raw_text = """Кони мчатся по буграм,
Топчут снег глубокий...
Вот в сторонке божий храм
Виден одинокий.
Вдруг метелица кругом;
Снег валит клоками;
Черный вран, свистя крылом,
Вьется над санями;
Вещий стон гласит печаль!
Кони торопливы
Чутко смотрят в темну даль,
Воздымая гривы...
Жуковский.

В конце 1811 года, в эпоху нам достопамятную, жил в своем поместье Ненарадове добрый Гаврила Гаврилович Р **. Он славился во всей округе гостеприимством и радушием; соседи поминутно ездили к нему поесть, попить, поиграть по пяти копеек в бостон с его женою, а некоторые для того, чтоб поглядеть на дочку их, Марью Гавриловну, стройную, бледную и семнадцатилетнюю девицу. Она считалась богатой невестою, и многие прочили ее за себя или за сыновей.
Марья Гавриловна была воспитана на французских романах и следственно была влюблена. Предмет, избранный ею, был бедный армейский прапорщик, находившийся в отпуску в своей деревне. Само по себе разумеется, что молодой человек пылал равною страстию, и что родители его любезной, заметя их взаимную склонность, запретили дочери о нем и думать, а его принимали хуже, нежели отставного заседателя.
Наши любовники были в переписке, и всякий день видались наедине в сосновой роще или у старой часовни. Там они клялися друг другу в вечной любви, сетовали на судьбу и делали различные предположения. Переписываясь и разговаривая таким образом, они (что весьма естественно) дошли до следующего рассуждения: если мы друг без друга дышать не можем, а воля жестоких родителей препятствует нашему благополучию, то нельзя ли нам будет обойтись без нее? Разумеется, что эта счастливая мысль пришла сперва в голову молодому человеку и что она весьма понравилась романическому воображению Марьи Гавриловны.
Наступила зима и прекратила их свидания, но переписка сделалась тем живее. Владимир Николаевич в каждом письме умолял ее предаться ему, венчаться тайно, скрываться несколько времени, броситься потом к ногам родителей, которые конечно будут тронуты наконец героическим постоянством и несчастием любовников и скажут им непременно: «Дети! придите в наши объятия».
Марья Гавриловна долго колебалась; множество планов побега было отвергнуто. Наконец она согласилась: в назначенный день она должна была не ужинать и удалиться в свою комнату под предлогом головной боли, Девушка ее была в заговоре; обе они должны были выйти в сад через заднее крыльцо, за садом найти готовые сани, садиться в них и ехать за пять верст от Ненарадова в село Жадрино, прямо в церковь, где уж Владимир должен был их ожидать.
Накануне решительного дня Марья Гавриловна не спала всю ночь; она укладывалась, увязывала белье и платье, написала длинное письмо к одной чувствительной барышне, ее подруге, другое к своим родителям. Она прощалась с ними в самых трогательных выражениях, извиняла свой проступок неодолимою силою страсти и оканчивала тем, что блаженнейшею минутою жизни почтет она ту, когда позволено будет ей броситься к ногам дражайших ее родителей. Запечатав оба письма тульской печаткою, на которой изображены были два пылающие сердца с приличной надписью, она бросилась на постель перед самым рассветом и задремала; но и тут ужасные мечтания поминутно ее пробуждали. То казалось ей, что в самую минуту, как она садилась в сани, чтоб ехать венчаться, отец ее останавливал ее, с мучительной быстротою тащил ее по снегу и бросал в темное, бездонное подземелие... и она летела стремглав с неизъяснимым замиранием сердца; то видела она Владимира, лежащего на траве, бледного, окровавленного. Он, умирая, молил ее пронзительным голосом поспешить с ним обвенчаться... другие безобразные, бессмысленные видения неслись перед нею одно за другим. Наконец она встала, бледнее обыкновенного и с непритворной головною болью. Отец и мать заметили ее беспокойство; их нежная заботливость и беспрестанные вопросы: что с тобою, Маша? не больна ли ты, Маша? — раздирали ее сердце. Она старалась их успокоить, казаться веселою, и не могла. Наступил вечер. Мысль, что уже в последний раз провожает она день посреди своего семейства, стесняла ее сердце. Она была чуть жива; она втайне прощалась со всеми особами, со всеми предметами, ее окружавшими.
Подали ужинать; сердце ее сильно забилось. Дрожащим голосом объявила она, что ей ужинать не хочется, и стала прощаться с отцом и матерью. Они ее поцеловали и, по обыкновению, благословили: она чуть не заплакала. Пришед в свою комнату, она кинулась в кресла и залилась слезами. Девушка уговаривала ее успокоиться и ободриться. Все было готово. Через полчаса Маша должна была навсегда оставить родительский дом, свою комнату, тихую девическую жизнь... На дворе была метель; ветер выл, ставни тряслись и стучали; всё казалось ей угрозой и печальным предзнаменованием. Скоро в доме всё утихло и заснуло. Маша окуталась шалью, надела теплый капот, взяла в руки шкатулку свою и вышла на заднее крыльцо. Служанка несла за нею два узла. Они сошли в сад. Метель не утихала; ветер дул навстречу, как будто силясь остановить молодую преступницу. Они насилу дошли до конца сада. На дороге сани дожидались их. Лошади, прозябнув, не стояли на месте; кучер Владимира расхаживал перед оглоблями, удерживая ретивых. Он помог барышне и ее девушке усесться и уложить узлы и шкатулку, взял вожжи, и лошади полетели. Поручив барышню попечению судьбы и искусству Терешки кучера, обратимся к молодому нашему любовнику.
Целый день Владимир был в разъезде. Утром был он у жадринского священника; насилу с ним уговорился; потом поехал искать свидетелей между соседними помещиками. Первый, к кому явился он, отставной сорокалетний корнет Дравин, согласился с охотою. Это приключение, уверял он, напоминало ему прежнее время и гусарские проказы. Он уговорил Владимира остаться у него отобедать и уверил его, что за другими двумя свидетелями дело не станет. В самом деле, тотчас после обеда явились землемер Шмит в усах и шпорах, и сын капитан-исправника, мальчик лет шестнадцати, недавно поступивший в уланы. Они не только приняли предложение Владимира, но даже клялись ему в готовности жертвовать для него жизнию. Владимир обнял их с восторгом и поехал домой приготовляться.
Уже давно смеркалось. Он отправил своего надежного Терешку в Ненарадово с своею тройкою и с подробным, обстоятельным наказом, а для себя велел заложить маленькие сани в одну лошадь, и один без кучера отправился в Жадрино, куда часа через два должна была приехать и Марья Гавриловна. Дорога была ему знакома, а езды всего двадцать минут.
Но едва Владимир выехал за околицу в поле, как поднялся ветер и сделалась такая метель, что он ничего не взвидел. В одну минуту дорогу занесло; окрестность исчезла во мгле мутной и желтоватой, сквозь которую летели белые хлопья снегу; небо слилося с землею. Владимир очутился в поле и напрасно хотел снова попасть на дорогу; лошадь ступала наудачу и поминутно то взъезжала на сугроб, то проваливалась в яму; сани поминутно опрокидывались. Владимир старался только не потерять настоящего направления. Но ему казалось, что уже прошло более получаса, а он не доезжал еще до Жадринской рощи. Прошло еще около десяти минут; рощи всё было не видать. Владимир ехал полем, пересеченным глубокими оврагами. Метель не утихала, небо не прояснялось. Лошадь начинала уставать, а с него пот катился градом, несмотря на то, что он поминутно был по пояс в снегу.
Наконец он увидел, что едет не в ту сторону. Владимир остановился: начал думать, припоминать, соображать — и уверился, что должно было взять ему вправо. Он поехал вправо. Лошадь его чуть ступала. Уже более часа был он в дороге. Жадрино должно было быть недалеко. Но он ехал, ехал, а полю не было конца. Всё сугробы да овраги; поминутно сани опрокидывались, поминутно он их подымал. Время шло; Владимир начинал сильно беспокоиться.
Наконец в стороне что-то стало чернеть. Владимир поворотил туда. Приближаясь, увидел он рощу. Слава богу, подумал он, теперь близко. Он поехал около рощи, надеясь тотчас попасть на знакомую дорогу или объехать рощу кругом: Жадрино находилось тотчас за нею. Скоро нашел он дорогу и въехал во мрак дерев, обнаженных зимою. Ветер не мог тут свирепствовать; дорога была гладкая; лошадь ободрилась, и Владимир успокоился.
Но он ехал, ехал, а Жадрина было не видать; роще не было конца. Владимир с ужасом увидел, что он заехал в незнакомый лес. Отчаяние овладело им. Он ударил по лошади; бедное животное пошло было рысью, но скоро стало приставать и через четверть часа пошло шагом, несмотря на все усилия несчастного Владимира.
Мало-помалу деревья начали редеть, и Владимир выехал из лесу; Жадрина было не видать. Должно было быть около полуночи. Слезы брызнули из глаз его; он поехал наудачу. Погода утихла, тучи расходились, перед ним лежала равнина, устланная белым волнистым ковром. Ночь была довольно ясна. Он увидел невдалеке деревушку, состоящую из четырех или пяти дворов. Владимир поехал к ней. У первой избушки он выпрыгнул из саней, подбежал к окну и стал стучаться. Через несколько минут деревянный ставень поднялся, и старик высунул свою седую бороду. «Что те надо?» — «Далеко ли Жадрино?» — «Жадрино-то далеко ли?» — «Да, да! Далеко ли?» — «Недалече; верст десяток будет». При сем ответе Владимир схватил себя за волосы и остался недвижим, как человек, приговоренный к смерти.
«А отколе ты?» — продолжал старик. Владимир не имел духа отвечать на вопросы. «Можешь ли ты, старик, — сказал он, — достать мне лошадей до Жадрина?» — «Каки у нас лошади», — отвечал мужик. «Да не могу ли взять хоть проводника? Я заплачу, сколько ему будет угодно». — «Постой, — сказал старик, опуская ставень, — я те сына вышлю; он те проводит». Владимир стал дожидаться. Не прошло минуты, он опять начал стучаться. Ставень поднялся, борода показалась. «Что те надо?» — «Что ж твой сын?» — «Сейчас выдет, обувается. Али ты прозяб? взойди погреться». — «Благодарю, высылай скорее сына».
Ворота заскрыпели; парень вышел с дубиною и пошел вперед, то указывая, то отыскивая дорогу, занесенную снеговыми сугробами. «Который час?» — спросил его Владимир. — «Да уж скоро рассвенет», — отвечал молодой мужик. Владимир не говорил уже ни слова.
Пели петухи, и было уже светло, как достигли они Жадрина. Церковь была заперта. Владимир заплатил проводнику и поехал на двор к священнику. На дворе тройки его не было. Какое известие ожидало его!
Но возвратимся к добрым ненарадовским помещикам и посмотрим, что-то у них делается.
А ничего."""
raw_text = raw_text.replace('\ufeff', '')
raw_text = re.sub(r"[^\w\s]", "", raw_text)
raw_text = re.sub(r"[^а-яА-Я\s]", "", raw_text)
raw_text = raw_text.lower()
words = raw_text.split()


# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(words)
vocab_size = len(vocab)
print(vocab_size)

word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

data = []
for i in range(2, len(words) - 2):
    context = [words[i - 2], words[i - 1],
               words[i + 1], words[i + 2]]
    target = words[i]
    data.append((context, target))

900


In [4]:
data

[(['кони', 'мчатся', 'буграм', 'топчут'], 'по'),
 (['мчатся', 'по', 'топчут', 'снег'], 'буграм'),
 (['по', 'буграм', 'снег', 'глубокий'], 'топчут'),
 (['буграм', 'топчут', 'глубокий', 'вот'], 'снег'),
 (['топчут', 'снег', 'вот', 'в'], 'глубокий'),
 (['снег', 'глубокий', 'в', 'сторонке'], 'вот'),
 (['глубокий', 'вот', 'сторонке', 'божий'], 'в'),
 (['вот', 'в', 'божий', 'храм'], 'сторонке'),
 (['в', 'сторонке', 'храм', 'виден'], 'божий'),
 (['сторонке', 'божий', 'виден', 'одинокий'], 'храм'),
 (['божий', 'храм', 'одинокий', 'вдруг'], 'виден'),
 (['храм', 'виден', 'вдруг', 'метелица'], 'одинокий'),
 (['виден', 'одинокий', 'метелица', 'кругом'], 'вдруг'),
 (['одинокий', 'вдруг', 'кругом', 'снег'], 'метелица'),
 (['вдруг', 'метелица', 'снег', 'валит'], 'кругом'),
 (['метелица', 'кругом', 'валит', 'клоками'], 'снег'),
 (['кругом', 'снег', 'клоками', 'черный'], 'валит'),
 (['снег', 'валит', 'черный', 'вран'], 'клоками'),
 (['валит', 'клоками', 'вран', 'свистя'], 'черный'),
 (['клоками', 'черн

In [5]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        #out: 1 x emdedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()
        
        #out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)
        

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]]).to(device)
        return self.embeddings(word).view(1,-1)


model = CBOW(vocab_size, EMDEDDING_DIM)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [6]:
model.to(device)
model

CBOW(
  (embeddings): Embedding(900, 10)
  (linear1): Linear(in_features=10, out_features=128, bias=True)
  (activation_function1): ReLU()
  (linear2): Linear(in_features=128, out_features=900, bias=True)
  (activation_function2): LogSoftmax(dim=-1)
)

In [7]:
#TRAINING
for epoch in tqdm(range(50)):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)  

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]).to(device))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

100%|██████████| 50/50 [00:56<00:00,  1.13s/it]


In [14]:
#TESTING
context = ['одинокий', 'вдруг', 'кругом', 'снег']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: {" ".join(words)}\n')
print(f'Context: {context}\n')
print(f'Prediction: {ix_to_word[torch.argmax(a[0]).item()]}')
print(model.get_word_emdedding('снег'))

Raw text: кони мчатся по буграм топчут снег глубокий вот в сторонке божий храм виден одинокий вдруг метелица кругом снег валит клоками черный вран свистя крылом вьется над санями вещий стон гласит печаль кони торопливы чутко смотрят в темну даль воздымая гривы жуковский в конце года в эпоху нам достопамятную жил в своем поместье ненарадове добрый гаврила гаврилович р он славился во всей округе гостеприимством и радушием соседи поминутно ездили к нему поесть попить поиграть по пяти копеек в бостон с его женою а некоторые для того чтоб поглядеть на дочку их марью гавриловну стройную бледную и семнадцатилетнюю девицу она считалась богатой невестою и многие прочили ее за себя или за сыновей марья гавриловна была воспитана на французских романах и следственно была влюблена предмет избранный ею был бедный армейский прапорщик находившийся в отпуску в своей деревне само по себе разумеется что молодой человек пылал равною страстию и что родители его любезной заметя их взаимную склонность запрет

In [9]:
word_vectors = model.embeddings.weight.data

In [10]:
# Определяем функцию для вычисления косинусного сходства между двумя векторами
def cosine_similarity(v1, v2):
    dot_product = torch.dot(v1, v2)
    norm_v1 = torch.norm(v1)
    norm_v2 = torch.norm(v2)
    return dot_product / (norm_v1 * norm_v2)

# Определяем функцию для поиска наиболее похожих слов на заданное слово
def find_similar_words(word, word_vectors, idx2word, top_k=5):
    word_idx = word_to_ix[word]
    word_vector = word_vectors[word_idx]
    similarities = []
    for i, vector in enumerate(word_vectors):
        if i == word_idx:
            continue
        similarity = cosine_similarity(word_vector, vector)
        similarities.append((i, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    similar_words = [idx2word[i] for i, _ in similarities[:top_k]]
    return similar_words

# Пробуем найти похожие слова на слово "война"
similar_words = find_similar_words("снег", word_vectors, ix_to_word)
print(f"Слова, похожие на слово 'снег': {', '.join(similar_words)}")

Слова, похожие на слово 'снег': склонность, вдруг, деревья, искать, лес
